# Time Series Analysis

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

## a) Exploration

### i)

### ii)

### iii)

## b) Investigation
### i)

### ii)

### iii)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

### iv)

In [ ]:
from statsmodels.tsa.seasonal import STL

## c) Forecasting


### i)

In [ ]:
from sktime.forecasting.arima import StatsModelsARIMA
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error